# 1. 패키지 설치

- 3.4와 코드는 동일함. data retrieval에서 활용되는 index 이름만 다름

In [1]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [3.4 LangChain을 활용한 Vector Database 변경 (Chroma ➡️ Pinecone)](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.4%20LangChain%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20Vector%20Database%20%EB%B3%80%EA%B2%BD%20(Chroma%20%E2%9E%A1%EF%B8%8F%20Pinecone).ipynb)와 동일함

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
document_list[52]

Document(page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [5]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


# 데이터를 추가할 때는 `from_documents()` 데이터를 추가한 이후에는 `from_existing_index()`를 사용합니다
# database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)


/Users/jasonkang/.pyenv/versions/3.10.12/envs/inflearn-llm-application/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
query = '연봉 5천만원인 직장인의 종합소득세는?'

# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [7]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n다. 삭제<2014. 12. 23.>\n\n라. 그 밖의 기타소득에 대해서는 100분의 20\

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [10]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
ai_message = qa_chain.invoke({"query": query})

In [12]:
ai_message

{'query': '연봉 5천만원인 직장인의 종합소득세는?',
 'result': '연봉 5천만원인 직장인의 종합소득세는 기본세율을 적용받습니다. 기본세율은 소득 구간에 따라 다르며, 정확한 세액은 소득세율표를 참조해야 합니다. 추가적인 공제 항목 등에 따라 최종 세액이 달라질 수 있습니다.'}